## library import &  data load

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [87]:
/pwd

'c:\\Users\\wq240\\Project\\case\\KoSimCSE\\new'

In [ ]:
# C:\Users\wq240\Project\case\KoSimCSE\판정구분.csv

In [90]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('../판정구분.csv', encoding='cp949')
case = pd.read_csv('../case_up.csv', encoding='cp949')

C:\Users\wq240\AppData\Local\Temp\ipykernel_34504\2493224557.py:3: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  case = pd.read_csv('../case_up.csv', encoding='cp949')


In [93]:
df.sample(1)

,보상파일번호,판정회차,사고번호,판정일,판정결재일,판정진행상태,사고진행상태,수출자,보험종목,수입국,수입자,은행,수출보증금액통화,수출보증금액,결제금액통화,결제금액,사고금액통화,사고금액,사고접수일자,보험금청구일,판정구분,판정금액통화,판정금액,판정사유
15144,2012120364500040,2,111022750,2012.08.29,2012.08.29,보상판정 결재완료,사후관리인수인계,(주)씨에이원,단기수출보험(일반수출),미국,GLOBAL PARTS SUPPLY INC,NaN,USD,"14,863.32",NaN,0,USD,"14,863.32",2012-05-17,2012.07.06,지급,USD,"14,863.32",지급 판정


In [10]:
exemption_df['판정사유'].value_counts()

판정사유
보험계약자의 고의 또는 과실로 인하여 발생한 손실             979
연속수출                                    734
보상한도를 초과하는 손실                           426
보험관계의 성립                                397
주의의무 해태로 인한 손실가중                        385
신용보증조건 위반                               286
신용보증관계의 성립                              236
권리보전의무 해태로 인한 손실가중                      231
적용대상 수출거래                               215
변제충당                                    141
보험계약의 해지(고지의무, 내용변경, 보험료 미납)            120
보증채무의 범위                                 44
신용보증대상 수출거래                              42
사고발생통지 의무                                42
보험계약이 공사가 책임 지울 수 없는 사유로 무효 실효 해제 해지     40
손실방지 경감의무                                34
보험증권 상 특약사항 해태로 인한 손실가중                  23
물품의 멸실, 훼손 또는 기타 물품에 대해 발생한 손실           16
본지사거래에서 신용위험으로 인하여 발생한 손실                14
보험책임 개시일전에 발생한 손실                        14
무신용장방식 거래에서 수출계약의 주요 사항 위반               14
금융계약상의 의무사항 불이행                          13
지시에 따를 의무                  

In [9]:

print('\n📌 판정사유 기본 정보')
print('-' * 40)
print(f'전체 행 수: {len(df):,}건')
print(f'판정사유 결측값: {df["판정사유"].isna().sum():,}건')
print(f'판정사유 유효값: {df["판정사유"].notna().sum():,}건')

# === 판정사유 value_counts (상위 20개) ===
print('\n📊 판정사유 상위 20개')
print('-' * 40)
print(df['판정사유'].value_counts().head(20))

# === 면책 케이스의 판정사유 분석 ===
exemption_df = df[df['판정구분'] == '면책']
print('\n🛡️ 면책 케이스 판정사유 분석')
print('-' * 40)
print(f'면책 케이스 총 수: {len(exemption_df):,}건')
print('\n면책 케이스의 판정사유 (상위 15개):')
print(exemption_df['판정사유'].value_counts().head(15))

# === 지급 케이스의 판정사유 분석 ===
payment_df = df[df['판정구분'] == '지급']
print('\n💰 지급 케이스 판정사유 분석')
print('-' * 40)
print(f'지급 케이스 총 수: {len(payment_df):,}건')
print('\n지급 케이스의 판정사유 (상위 10개):')
print(payment_df['판정사유'].value_counts().head(10))


📌 판정사유 기본 정보
----------------------------------------
전체 행 수: 20,618건
판정사유 결측값: 0건
판정사유 유효값: 20,618건

📊 판정사유 상위 20개
----------------------------------------
판정사유
지급 판정                                   15764
보험계약자의 고의 또는 과실로 인하여 발생한 손실               979
연속수출                                      734
보상한도를 초과하는 손실                             426
보험관계의 성립                                  397
주의의무 해태로 인한 손실가중                          385
신용보증조건 위반                                 286
신용보증관계의 성립                                236
권리보전의무 해태로 인한 손실가중                        231
적용대상 수출거래                                 215
지급유예 판정                                   174
변제충당                                      141
보험계약의 해지(고지의무, 내용변경, 보험료 미납)              120
보험계약자와 수출계약상대방간에 분쟁발생                      84
지급할 보험금을 산정하기 위하여 장기간이 소요되는 경우             70
보증채무의 범위                                   44
사고발생통지 의무                                  42
신용보증대상 수출거래                                42
보험계약이 공사가

In [11]:
# 🛡️ 면책 사례 중 의미 있는 사고설명 필터링
exemption_df = df[df['판정구분'] == '면책']
meaningful_exemption = exemption_df[
    (exemption_df['사고설명'].notna()) &
    (exemption_df['사고설명'].str.len() > 20) &
    (~exemption_df['사고설명'].str.contains('설명없음|첨부파일참고|해당없음|참고', na=False, case=False))
]

print('\n🧾 의미 있는 사고설명 있는 면책 사례')
print('-' * 50)
print(f'총 {len(meaningful_exemption):,}건의 사고설명이 의미 있음')

# 🔍 판정사유별 사고설명 샘플 확인
top_exemption_reasons = exemption_df['판정사유'].value_counts().head(5)
print('\n📌 면책 주요 사유별 사고설명 샘플')
print('-' * 50)

for reason in top_exemption_reasons.index:
    print(f'\n🔸 판정사유: {reason}')
    samples = meaningful_exemption[meaningful_exemption['판정사유'] == reason]['사고설명'].head(3)
    for i, desc in enumerate(samples, start=1):
        if pd.notna(desc) and len(str(desc)) > 20:
            print(f'  {i}. {desc}')


🧾 의미 있는 사고설명 있는 면책 사례
--------------------------------------------------
총 2,863건의 사고설명이 의미 있음

📌 면책 주요 사유별 사고설명 샘플
--------------------------------------------------

🔸 판정사유: 보험계약자의 고의 또는 과실로 인하여 발생한 손실
  1. o (주)탑글로벌파츠 대 아랍에미리트 연합 HEAVY VEHICLES & SPARE PARTS TR.EST 단기수출보험(단체보험) 사고건
  - 사고금액 : USD7,321.70, 수입자 R급 기등록('25.3.27자)
  - 경합수출자 및 동일수입자 : 없음.  끝.
  2. ㅇ (주)경아로지스틱스 대 아랍에미리트연합 BLUE FIRE GOODS WHOLESALERS LLC 수출신용보증(선적후) 및 연계보험 사고건
  - 사고금액 : USD 399,930.-
  - 경합수출자 1개사 (주)폴맥 앞 단기수출보험 경합한도 USD3백만, 만기미도래금액 USD1백만 존재
  - 수입자 R급 등록 완료('24.7.15자)하였으며, 동일대표 수입자는 없음.  끝.
  3. ㅇ (주)타라유통 대 칠레 PAPELERA DIMAR S A 간 단기수출보험(일반수출) 사고 건
 - 총 사고금액 USD 338,974.3 / 부보율 97.5%  
 - 수입자 R급 평가 완료('24.1.31자)
 - 경합수출자 없음

🔸 판정사유: 연속수출
  1. o (주)나현텍스타일 대 프랑스 AVITEX(321-320248) 플러스보험 사고건
  - 수출입자간 총 사고금액 : USD152,970.65 (단체보험 : USD52,644.25 / 플러스보험 : USD100,326.40)
  - 동일대표수입자 : MAILLE LOVE(321-108847), MAILLE WORLD(321-108893) 2개사 R급 기등록
  - 경합수출자 : (주)제이제이텍스타일 등 총 5개사 플러스보험 한도 각 USD10만 보유.  끝.
  2. 

In [17]:
df['상품분류그룹명'].value_counts()

상품분류그룹명
의류 및 직물류             3909
무기 및 유기화학제품          2630
수출 품목 그룹 정보 미제공      2588
전기 및 전자제품            1687
기계류                  1635
금속, 비금속류             1560
고무, 가죽               1533
운송장비 및 부품            1486
기타                   1018
목재와 펄프, 지물류           985
농수산물, 식료품             729
정밀기기, 시계, 악기, 무기류     318
도기, 유리, 보석류           306
완구, 운동용구, 잡품          141
소금, 광물류                63
제조설비                   13
기타산업설비                 12
발전설비                    2
해외엔지니어링 활동              2
통신설비                    1
Name: count, dtype: int64

In [16]:
df['상품분류명'].nunique()

823

In [94]:
case.sample(1)

,보상파일번호,사고접수일자,사고접수확정일자,사고번호,사고유형명,사고진행상태,사고등록상태명,접수사고유형확정,사고설명,사고금액통화,외화사고접수금액,미화사고접수금액,원화사고접수금액,판정통화코드_ZZ067,외화합계판정금액,미화합계판정금액,원화합계판정금액,외화판정금액,미화판정금액,원화판정금액,통화기준일자_x,판정사유순번,심사항목명,상품심사항목명,사고접수일자.1,외화사고금액,미화사고금액,원화사고금액,외화보험가액,미화보험가액,원화보험가액,외화보험금액,미화보험금액,원화보험금액,수출자대상자번호,수입자대상자번호,상품분류명,상품분류그룹명,보험성립일자,청약번호,청약변경회차,수출보험계약번호,수출보험계약변경회차,수출보험계약내역순번,보험종목,상품비고내용,상세사고보험상품,지급기준일자,신속보상대상여부,손실분담사고여부,매입보증연계보험가입여부,부보율,결제방법,결제방법설명,결제조건,향후결제전망,수출자,수입자명,수출자국가,수입국
13849,2015120351210060,2015-12-08,2015-12-10,111030933,신용위험-지급지체,전액회수종결,채권종결 결재완료,신용위험-지급지체,"- 사고금액 : EUR 3,240,000 (보상한도 USD 3,000,000)\n-...",EUR,1346400.0,1527490.8,1678260672,EUR,1253732.93,1422360.0,1562753022,1253732.93,1422360.0,1562753022,20170809.0,1.0,수입자 신용도,수입자 신용도_미사용,2015-12-08,1346400.0,1527490.8,1678260672,1346400.0,1527490.8,1678260000,1279080.0,1451116.26,1594347000,3100000295,6000645165,리튬폴리머축전지,전기 및 전자제품,20150316.0,1.120150e+11,0.0,112015318151,0.0,1.0,단기수출보험(일반수출),NaN,일반,20170809.0,N,N,N,95.0,O/A(T/T 포함),결제방법(인수),Days After B/L Date,판단불가,(주)엘지화학,INABENSA ELECTRICAL TIANJIN CO LTD,대한민국,중국


In [ ]:
df.merge(case, on=['보상파일번호', '사고번호', '사고금액', '외화판정금액'], how='left')

In [ ]:
# 원본 사고보상판정 테이블에서 수출상품군 파악이 어려운 개수
# 총 25550 중에 5859개 => 1/5은 상품군으로 파악이 어려움
# 업종으로 대체하는 것도 크게 의미없을 것 같음, 해당 업종에 해당하는 상품만으로 수출입을 하는게 아닐 것이기 때문문
case[(case['상품분류명'] == '기타') & (case['상품분류그룹명'] == '기타')].shape

(5859, 60)

In [72]:
case.columns

Index(['보상파일번호', '사고접수일자', '사고접수확정일자', '사고번호', '사고유형명', '사고진행상태', '사고등록상태명',
       '접수사고유형확정', '사고설명', '사고통화코드_ZZ067', '외화사고접수금액', '미화사고접수금액', '원화사고접수금액',
       '판정통화코드_ZZ067', '외화합계판정금액', '미화합계판정금액', '원화합계판정금액', '외화판정금액', '미화판정금액',
       '원화판정금액', '통화기준일자_x', '판정사유순번', '심사항목명', '상품심사항목명', '사고접수일자.1',
       '외화사고금액', '미화사고금액', '원화사고금액', '외화보험가액', '미화보험가액', '원화보험가액', '외화보험금액',
       '미화보험금액', '원화보험금액', '수출자대상자번호', '수입자대상자번호', '상품분류명', '상품분류그룹명',
       '보험성립일자', '청약번호', '청약변경회차', '수출보험계약번호', '수출보험계약변경회차', '수출보험계약내역순번',
       '상품명', '상품비고내용', '상세사고보험상품', '지급기준일자', '신속보상대상여부', '손실분담사고여부',
       '매입보증연계보험가입여부', '부보율', '결제방법', '결제방법설명', '결제조건', '향후결제전망', '수출자명',
       '수입자명', '수출자국가', '수입자국가'],
      dtype='object')

In [101]:
df.sample(1)

,보상파일번호,판정회차,사고번호,판정일,판정결재일,판정진행상태,사고진행상태,수출자,보험종목,수입국,수입자,은행,수출보증금액통화,수출보증금액,결제금액통화,결제금액,사고금액통화,외화사고금액,사고접수일자,보험금청구일,판정구분,판정금액통화,외화판정금액,판정사유
9502,2016120351350019,1,111032976,2017.04.24,2017.04.25,보상판정 결재완료,사후관리인수인계,한화에어로스페이스(주),단기수출보험(일반수출),인도,AUTOCOP INDIA PRIVATE LIMITED,NaN,USD,72778.0,NaN,0,USD,46299.0,2016-09-21,2016.12.21,지급,USD,43984.05,지급 판정


In [120]:
case.sample(1)

,보상파일번호,사고접수일자,사고접수확정일자,사고번호,사고유형명,사고진행상태,사고등록상태명,접수사고유형확정,사고설명,사고금액통화,외화사고접수금액,미화사고접수금액,원화사고접수금액,판정통화코드_ZZ067,외화합계판정금액,미화합계판정금액,원화합계판정금액,외화판정금액,미화판정금액,원화판정금액,통화기준일자_x,판정사유순번,심사항목명,상품심사항목명,사고접수일자.1,외화사고금액,미화사고금액,원화사고금액,외화보험가액,미화보험가액,원화보험가액,외화보험금액,미화보험금액,원화보험금액,수출자대상자번호,수입자대상자번호,상품분류명,상품분류그룹명,보험성립일자,청약번호,청약변경회차,수출보험계약번호,수출보험계약변경회차,수출보험계약내역순번,보험종목,상품비고내용,상세사고보험상품,지급기준일자,신속보상대상여부,손실분담사고여부,매입보증연계보험가입여부,부보율,결제방법,결제방법설명,결제조건,향후결제전망,수출자,수입자명,수출자국가,수입국
1765,201012041ZZZ0068,2010-02-11,2010-02-11,151013643,이자연체,사후관리인수인계,사고접수 결재완료,이자연체,기업은행 원주지점으로부터 사고접수,KRW,180000000.0,142664.65,180000000,KRW,183313970.0,145291.24,183313970,180000000.0,142664.65,180000000,20100412.0,1.0,수입자 신용도,수입자 신용도_미사용,2010-02-11,180000000.0,142664.65,180000000,180000000.0,142664.65,180000000,180000000.0,142664.65,180000000,3100117298,NaN,기타,기타,20090518.0,1.520091e+11,0.0,152009672395,0.0,1.0,수출신용보증(선적전),NaN,일반,20100412.0,N,N,N,100.0,NaN,NaN,NaN,판단불가,맥스테크(주),NaN,대한민국,NaN


In [106]:
df.shape

(23296, 24)

In [107]:
df.drop_duplicates(inplace=True)
df.shape

(23296, 24)

In [108]:
case.drop_duplicates(inplace=True)
case.shape

(25476, 60)

In [155]:
# df : (20706, 32)
dd = df.merge(case[['보상파일번호', '사고번호', '사고금액통화', '외화사고금액', '외화판정금액','미화판정금액','원화판정금액', '미화사고금액', '원화사고금액', '상품분류명',	'상품분류그룹명', '사고유형명', '접수사고유형확정', '사고설명','부보율', '결제방법', '결제방법설명', '결제조건', '향후결제전망', '수출자국가']], on=['보상파일번호', '사고번호', '외화사고금액', '외화판정금액', '사고금액통화'])
dd.shape

(21774, 39)

In [156]:
dd.drop_duplicates(inplace=True)
dd.shape

(20618, 39)

In [ ]:
'사고유형명', '접수사고유형확정', '사고설명','부보율', '결제방법', '결제방법설명', '결제조건', '향후결제전망', '수출자국가'

In [154]:
case.columns

Index(['보상파일번호', '사고접수일자', '사고접수확정일자', '사고번호', '사고유형명', '사고진행상태', '사고등록상태명',
       '접수사고유형확정', '사고설명', '사고금액통화', '외화사고접수금액', '미화사고접수금액', '원화사고접수금액',
       '판정통화코드_ZZ067', '외화합계판정금액', '미화합계판정금액', '원화합계판정금액', '외화판정금액', '미화판정금액',
       '원화판정금액', '통화기준일자_x', '판정사유순번', '심사항목명', '상품심사항목명', '사고접수일자.1',
       '외화사고금액', '미화사고금액', '원화사고금액', '외화보험가액', '미화보험가액', '원화보험가액', '외화보험금액',
       '미화보험금액', '원화보험금액', '수출자대상자번호', '수입자대상자번호', '상품분류명', '상품분류그룹명',
       '보험성립일자', '청약번호', '청약변경회차', '수출보험계약번호', '수출보험계약변경회차', '수출보험계약내역순번',
       '보험종목', '상품비고내용', '상세사고보험상품', '지급기준일자', '신속보상대상여부', '손실분담사고여부',
       '매입보증연계보험가입여부', '부보율', '결제방법', '결제방법설명', '결제조건', '향후결제전망', '수출자',
       '수입자명', '수출자국가', '수입국'],
      dtype='object')

In [153]:
dd.columns

Index(['보상파일번호', '판정회차', '사고번호', '판정일', '판정결재일', '판정진행상태', '사고진행상태', '수출자',
       '보험종목', '수입국', '수입자', '수출보증금액통화', '수출보증금액', '사고금액통화', '외화사고금액',
       '사고접수일자', '보험금청구일', '판정구분', '판정금액통화', '외화판정금액', '판정사유', '미화판정금액',
       '원화판정금액', '미화사고금액', '원화사고금액', '상품분류명', '상품분류그룹명'],
      dtype='object')

In [130]:
case.isna().mean()

보상파일번호          0.000000
사고접수일자          0.000000
사고접수확정일자        0.000039
사고번호            0.000000
사고유형명           0.000000
사고진행상태          0.000039
사고등록상태명         0.000000
접수사고유형확정        0.000000
사고설명            0.079408
사고금액통화          0.000000
외화사고접수금액        0.000000
미화사고접수금액        0.000000
원화사고접수금액        0.000000
판정통화코드_ZZ067    0.000000
외화합계판정금액        0.000000
미화합계판정금액        0.000000
원화합계판정금액        0.000000
외화판정금액          0.000000
미화판정금액          0.000000
원화판정금액          0.000000
통화기준일자_x        0.000079
판정사유순번          0.000000
심사항목명           0.000000
상품심사항목명         0.000000
사고접수일자.1        0.000000
외화사고금액          0.000000
미화사고금액          0.000000
원화사고금액          0.000000
외화보험가액          0.000000
미화보험가액          0.000000
원화보험가액          0.000000
외화보험금액          0.000000
미화보험금액          0.000000
원화보험금액          0.000000
수출자대상자번호        0.000000
수입자대상자번호        0.231473
상품분류명           0.128003
상품분류그룹명         0.128003
보험성립일자          0.000000
청약번호            0.108848


In [157]:
dd.drop_duplicates(subset=['보상파일번호', '사고번호', '외화사고금액', '외화판정금액','미화판정금액','원화판정금액', '미화사고금액', '원화사고금액', '판정구분', '판정회차', '판정사유', '사고금액통화']).shape

(20616, 39)

In [84]:
dd.drop_duplicates(subset=['보상파일번호', '사고번호', '외화사고금액', '외화판정금액','미화판정금액','원화판정금액', '미화사고금액', '원화사고금액', '판정회차', '판정사유']).shape

(20727, 38)

#### 중복 내용 확인용

In [67]:
df.groupby(['보상파일번호', '사고번호', '외화사고금액', '외화판정금액', '판정회차', '판정사유']).size().sort_values(ascending=False).head(20).reset_index(name='size').sort_values(by='size', ascending=False)

,보상파일번호,사고번호,외화사고금액,외화판정금액,판정회차,판정사유,size
0,2025122033600001,211000776,7442379,7442379,1,지급 판정,1
1,2010120344500001,111019727,2087,2087,2,"보험계약의 해지(고지의무, 내용변경, 보험료 미납)",1
2,2010120344500001,111019727,2087,2087,2,연속수출,1
3,2010120344500001,111019728,17578,17578,2,"보험계약의 해지(고지의무, 내용변경, 보험료 미납)",1
4,2010120344500001,111019728,17578,17578,2,연속수출,1
5,2010120344500001,111019729,4108,4108,2,"보험계약의 해지(고지의무, 내용변경, 보험료 미납)",1
6,2010120344500001,111019729,4108,4108,2,연속수출,1
7,2010120344500001,111019730,9740,9740,2,"보험계약의 해지(고지의무, 내용변경, 보험료 미납)",1
8,2010120344500001,111019730,9740,9740,2,연속수출,1
9,2010120344500001,111019731,1017,1017,2,"보험계약의 해지(고지의무, 내용변경, 보험료 미납)",1


In [42]:
df[(df['보상파일번호'] == '2015120351210003')].shape

(36, 32)

In [55]:
# 첨부파일 참조 텍스트 780개
df[(df['사고설명'].str.contains('첨부파일', na=False)) & (df['사고설명'].str.len() < 15)].shape

(780, 32)

In [40]:
# 판정 구분차이(판정회차로 판단가능), 같은 판정도 다른 사유 가짐
df[(df['보상파일번호'] == '2015120351210003') & (df['사고번호'] == 151017583)]

,보상파일번호,판정회차,사고번호,판정일,판정결재일,판정진행상태,사고진행상태,수출자,보험종목,수입국,수입자,수출보증금액통화,수출보증금액,결제금액통화,결제금액,사고금액통화,외화사고금액,사고접수일자,보험금청구일,판정구분,판정금액통화,외화판정금액,판정사유,사고유형명,사고등록상태명,접수사고유형확정,사고설명,결제방법,결제방법설명,결제조건,향후결제전망,수입자명
9807,2015120351210003,3,151017583,2024.04.04,2024.04.04,보상판정 결재완료,사후관리인수인계,(주)모뉴엘,수출신용보증(선적후),중국,CHINA NATIONAL BUILDING MATERIALS AND EQUIPMEN...,USD,977130,NaN,0,USD,977130,2015-01-05,2015.01.05,지급,USD,977130,지급 판정,신용위험-지급지체,사고접수 결재완료,신용위험-지급지체,- 첨부 사고발생통지서 참조,O/A(T/T 포함),결제방법(인수),Days From B/L Date,판단불가,CHINA NATIONAL BUILDING MATERIALS AND EQUIPMEN...
9811,2015120351210003,2,151017583,2017.02.20,2017.02.20,보상판정 결재완료,사후관리인수인계,(주)모뉴엘,수출신용보증(선적후),중국,CHINA NATIONAL BUILDING MATERIALS AND EQUIPMEN...,USD,977130,NaN,0,USD,977130,2015-01-05,2015.01.05,지급,USD,977130,지급 판정,신용위험-지급지체,사고접수 결재완료,신용위험-지급지체,- 첨부 사고발생통지서 참조,O/A(T/T 포함),결제방법(인수),Days From B/L Date,판단불가,CHINA NATIONAL BUILDING MATERIALS AND EQUIPMEN...
17851,2015120351210003,1,151017583,2015.01.23,2015.01.26,보상판정 결재완료,사후관리인수인계,(주)모뉴엘,수출신용보증(선적후),중국,CHINA NATIONAL BUILDING MATERIALS AND EQUIPMEN...,USD,977130,NaN,0,USD,977130,2015-01-05,2015.01.05,면책,USD,977130,신용보증관계의 성립,신용위험-지급지체,사고접수 결재완료,신용위험-지급지체,- 첨부 사고발생통지서 참조,O/A(T/T 포함),결제방법(인수),Days From B/L Date,판단불가,CHINA NATIONAL BUILDING MATERIALS AND EQUIPMEN...
17852,2015120351210003,1,151017583,2015.01.23,2015.01.26,보상판정 결재완료,사후관리인수인계,(주)모뉴엘,수출신용보증(선적후),중국,CHINA NATIONAL BUILDING MATERIALS AND EQUIPMEN...,USD,977130,NaN,0,USD,977130,2015-01-05,2015.01.05,면책,USD,977130,신용보증조건 위반,신용위험-지급지체,사고접수 결재완료,신용위험-지급지체,- 첨부 사고발생통지서 참조,O/A(T/T 포함),결제방법(인수),Days From B/L Date,판단불가,CHINA NATIONAL BUILDING MATERIALS AND EQUIPMEN...
17853,2015120351210003,1,151017583,2015.01.23,2015.01.26,보상판정 결재완료,사후관리인수인계,(주)모뉴엘,수출신용보증(선적후),중국,CHINA NATIONAL BUILDING MATERIALS AND EQUIPMEN...,USD,977130,NaN,0,USD,977130,2015-01-05,2015.01.05,면책,USD,977130,주의의무 해태로 인한 손실가중,신용위험-지급지체,사고접수 결재완료,신용위험-지급지체,- 첨부 사고발생통지서 참조,O/A(T/T 포함),결제방법(인수),Days From B/L Date,판단불가,CHINA NATIONAL BUILDING MATERIALS AND EQUIPMEN...


#### 금액 변수 추가
- 사고금액, 판정금액 기준 맞춰서 df(원본: test.csv)에 추가

In [100]:
# df['외화사고금액'] = df['외화사고금액'].str.replace(',', '').astype(float)
df['수출보증금액'] = df['수출보증금액'].str.replace(',', '').astype(float)
df['외화판정금액'] = df['외화판정금액'].str.replace(',', '').astype(float)

In [95]:
# case['외화사고금액'] = round(case['외화사고금액']).astype(int)
# case['외화판정금액'] = round(case['외화판정금액']).astype(int)

df.rename(columns={'사고금액': '외화사고금액', '판정금액': '외화판정금액'}, inplace=True)

In [35]:
case.rename(columns={'보상파일번호_x': '보상파일번호'}, inplace=True)

In [36]:
case['외화사고금액'] = round(case['외화사고금액']).astype(int)

In [19]:
case.groupby(['보상파일번호', '사고번호', '외화사고금액', '외화판정금액']).count()

사고접수일자  사고접수확정일자  사고유형명  \
보상파일번호           사고번호      외화사고금액    외화판정금액                               
2010120344500001 111019727 2087      2087            1         1      1   
                 111019728 17578     17578           1         1      1   
                 111019729 4108      4108            1         1      1   
                 111019730 9740      9740            1         1      1   
                 111019731 1017      1017            1         1      1   
...                                                ...       ...    ...   
202512041ZZZ0200 151022751 194400000 194400000       1         1      1   
202512041ZZZ0204 151022755 950000000 949948726       1         1      1   
202512041ZZZ0207 151022758 256500000 256500000       1         1      1   
202512041ZZZ0214 151022765 95000000  95000000        1         1      1   
2025122033600001 211000776 7442379   7442379         1         1      1   

                                                사고진행상태  사고등록상태명  접수사고유형확정  \
보상파일번호           사고번호      외화사고금액    외화판정금액                                 
2010120344500001 111019727 2087      2087            1        1         1   
                 111019728 17578     17578           1        1         1   
                 111019729 4108      4108            1        1         1   
                 111019730 9740      9740            1        1         1   
                 111019731 1017      1017            1        1         1   
...                                                ...      ...       ...   
202512041ZZZ0200 151022751 194400000 194400000       1        1         1   
202512041ZZZ0204 151022755 950000000 949948726       1        1         1   
202512041ZZZ0207 151022758 256500000 256500000       1        1         1   
202512041ZZZ0214 151022765 95000000  95000000        1        1         1   
2025122033600001 211000776 7442379   7442379         1        1         1   

                                                사고설명  사고통화코드_ZZ067  외화사고접수금액  \
보상파일번호           사고번호      외화사고금액    외화판정금액                                    
2010120344500001 111019727 2087      2087          1             1         1   
                 111019728 17578     17578         1             1         1   
                 111019729 4108      4108          1             1         1   
                 111019730 9740      9740          1             1         1   
                 111019731 1017      1017          1             1         1   
...                                              ...           ...       ...   
202512041ZZZ0200 151022751 194400000 194400000     1             1         1   
202512041ZZZ0204 151022755 950000000 949948726     1             1         1   
202512041ZZZ0207 151022758 256500000 256500000     1             1         1   
202512041ZZZ0214 151022765 95000000  95000000      1             1         1   
2025122033600001 211000776 7442379   7442379       1             1         1   

                                                미화사고접수금액  원화사고접수금액  \
보상파일번호           사고번호      외화사고금액    외화판정금액                          
2010120344500001 111019727 2087      2087              1         1   
                 111019728 17578     17578             1         1   
                 111019729 4108      4108              1         1   
                 111019730 9740      9740              1         1   
                 111019731 1017      1017              1         1   
...                                                  ...       ...   
202512041ZZZ0200 151022751 194400000 194400000         1         1   
202512041ZZZ0204 151022755 950000000 949948726         1         1   
202512041ZZZ0207 151022758 256500000 256500000         1         1   
202512041ZZZ0214 151022765 95000000  95000000          1         1   
2025122033600001 211000776 7442379   7442379           1         1   

                                                판정통화코드_ZZ067  외화합계판정금액  \
보상파일번호           사고번호      외화사고금액    외화판정금액 

In [17]:
df.groupby(['보상파일번호', '사고번호', '외화사고금액', '외화판정금액'])['판정회차'].count()

보상파일번호            사고번호       외화사고금액   외화판정금액 
2010120344500001  111019727  2087     2087       2
                  111019728  17578    17578      2
                  111019729  4108     4108       2
                  111019730  9740     9740       2
                  111019731  1017     1017       2
                                                ..
2025120379990009  111045371  20000    19000      1
2025120379990015  111045509  7322     2723       1
                                      4456       1
2025120379990023  111045611  7681009  7296958    1
2025122033600001  211000776  7442379  7442379    1
Name: 판정회차, Length: 18791, dtype: int64

In [24]:
df.외화사고금액.value_counts()

외화사고금액
977130     204
1265000    140
1490000    116
864864     115
100000      95
          ... 
363630       1
17920        1
92950        1
46660        1
479          1
Name: count, Length: 13130, dtype: int64

(18800, 36)

In [13]:
df.shape

(20706, 32)

In [14]:
df[df['보상파일번호'] == '2012120351210005']

,보상파일번호,판정회차,사고번호,판정일,판정결재일,판정진행상태,사고진행상태,수출자,보험종목,수입국,수입자,수출보증금액통화,수출보증금액,결제금액통화,결제금액,사고금액통화,사고금액,사고접수일자,보험금청구일,판정구분,판정금액통화,판정금액,판정사유,사고유형명,사고등록상태명,접수사고유형확정,사고설명,결제방법,결제방법설명,결제조건,향후결제전망,수입자명
13847,2012120351210005,1,111021941,2012.09.06,2012.09.10,보상판정 결재완료,사후관리인수인계,엘지전자(주),단기수출보험(글로벌네트워크),중국,BEIJING SHIYUANDA ELECTRONICS TECHNOLOGY CO LTD,CNY,0,NaN,0,CNY,6506,2011-12-30,2012.04.17,지급,CNY,6181,지급 판정,신용위험-지급지체,사고접수 결재완료,신용위험-지급지체,수입자 지급지체(첨부파일 참조),O/A(T/T 포함),결제방법(인수),Days After Invoice Date,판단불가,BEIJING SHIYUANDA ELEC TECH CO LTD
13848,2012120351210005,1,111021940,2012.09.06,2012.09.10,보상판정 결재완료,사후관리인수인계,엘지전자(주),단기수출보험(글로벌네트워크),중국,BEIJING SHIYUANDA ELECTRONICS TECHNOLOGY CO LTD,USD,0,NaN,0,USD,276201,2011-12-30,2012.04.17,지급,USD,262391,지급 판정,신용위험-지급지체,사고접수 결재완료,신용위험-지급지체,수입자 지급지체(첨부파일 참조),O/A(T/T 포함),결제방법(인수),Days After Invoice Date,판단불가,BEIJING SHIYUANDA ELEC TECH CO LTD


### 전처리

In [ ]:
'사고유형명', '접수사고유형확정', '사고설명','부보율', '결제방법', '결제방법설명', '결제조건', '향후결제전망', '수출자국가'

In [159]:
dd.sample(3)

,보상파일번호,판정회차,사고번호,판정일,판정결재일,판정진행상태,사고진행상태,수출자,보험종목,수입국,수입자,은행,수출보증금액통화,수출보증금액,결제금액통화,결제금액,사고금액통화,외화사고금액,사고접수일자,보험금청구일,판정구분,판정금액통화,외화판정금액,판정사유,미화판정금액,원화판정금액,미화사고금액,원화사고금액,상품분류명,상품분류그룹명,사고유형명,접수사고유형확정,사고설명,부보율,결제방법,결제방법설명,결제조건,향후결제전망,수출자국가
15311,2010120376570001,1,111019222,2010.06.08,2010.06.08,보상판정 결재완료,사후관리인수인계,미래글로발리미티드(주),단기수출보험(일반수출),남아프리카공화국,SOUTH AFRICAN COMPOUNDS PTY LTD,NaN,USD,43200.00,NaN,0,USD,43200.00,2010-03-08,2010.04.08,지급,USD,43200.00,지급 판정,43200.00,50055840,43200.00,50055840,오르토프탈산 디-2-에틸헥실,무기 및 유기화학제품,신용위험-지급지체,신용위험-지급지체,"수출신용보증(선적후) 사고접수건\nㅇ 사고금액 : U$43,200\nㅇ 사고사유 :...",100.0,D/A,결제방법(인수),Days From B/L Date,판단불가,대한민국
14344,2011120376370002,1,151014623,2011.05.26,2011.05.27,보상판정 결재완료,사후관리인수인계중,(주)광진상사,수출신용보증(선적후),리비아,EL MAYDAN COMPANY FOR EXPORT AND IMPORT CO LTD,NaN,USD,125113.86,NaN,0,USD,125113.86,2011-04-13,2011.05.17,지급,USD,125113.86,지급 판정,125113.86,140064966,125113.86,140064966,기타,운송장비 및 부품,비상위험-전쟁 등으로 인한 환거래 불능,비상위험-전쟁 등으로 인한 환거래 불능,리비아 소요사태로 인한 은행 폐쇄로 대금지급 불능,100.0,D/A,결제방법(인수),Days From B/L Date,판단불가,대한민국
18005,2015120364500016,1,111029492,2015.10.08,2015.10.13,보상판정 결재완료,면책종결,(주)지에스티아이엔씨,단기수출보험(일반수출),미국,SAMSUNG C AND T AMERICA INC,NaN,USD,82205.34,NaN,.0,USD,82205.34,2015-05-21,2015.06.03,면책,USD,82205.34,보험계약자의 고의 또는 과실로 인하여 발생한 손실,82205.34,90171037,82205.34,90171037,합성섬유제의 것,의류 및 직물류,신용위험-지급거절,신용위험-지급지체,첨부 사고발생통지서 참조,100.0,O/A(T/T 포함),결제방법(인수),Days After B/L Date,판단불가,대한민국


In [158]:
dd.판정진행상태.value_counts()

판정진행상태
보상판정 결재완료    20611
보상판정 등록중         7
Name: count, dtype: int64

In [138]:
dd[dd.판정사유.isna()].shape

(7, 30)

In [162]:
dd.isna().mean()

보상파일번호      0.000000
판정회차        0.000000
사고번호        0.000000
판정일         0.000000
판정결재일       0.000340
판정진행상태      0.000000
사고진행상태      0.000000
수출자         0.000000
보험종목        0.000000
수입국         0.009070
수입자         0.009070
수출보증금액통화    0.000000
수출보증금액      0.000000
사고금액통화      0.000000
외화사고금액      0.000000
사고접수일자      0.000000
보험금청구일      0.000000
판정구분        0.000000
판정금액통화      0.000000
외화판정금액      0.000000
판정사유        0.000340
미화판정금액      0.000000
원화판정금액      0.000000
미화사고금액      0.000000
원화사고금액      0.000000
상품분류명       0.125521
상품분류그룹명     0.125521
사고유형명       0.000000
접수사고유형확정    0.000000
사고설명        0.071830
부보율         0.000000
결제방법        0.042924
결제방법설명      0.047725
결제조건        0.142836
향후결제전망      0.000000
수출자국가       0.000679
dtype: float64

In [161]:
dd.drop(columns=['은행', '결제금액통화', '결제금액'], inplace=True)

In [163]:
dd['판정사유'] = dd['판정사유'].fillna('사유 없음')

In [164]:
# 품목 정보 미제공
dd['상품분류명'] = dd['상품분류명'].fillna('수출 품목 정보 미제공')
dd['상품분류그룹명'] = dd['상품분류그룹명'].fillna('수출 품목 그룹 정보 미제공')

In [144]:
dd['상품분류그룹명'].value_counts()

상품분류그룹명
의류 및 직물류             3909
무기 및 유기화학제품          2630
전기 및 전자제품            1687
기계류                  1635
금속, 비금속류             1560
고무, 가죽               1533
운송장비 및 부품            1486
기타                   1018
목재와 펄프, 지물류           985
농수산물, 식료품             729
정밀기기, 시계, 악기, 무기류     318
도기, 유리, 보석류           306
완구, 운동용구, 잡품          141
소금, 광물류                63
제조설비                   13
기타산업설비                 12
발전설비                    2
해외엔지니어링 활동              2
통신설비                    1
Name: count, dtype: int64

In [148]:
dd[dd['상품분류명'] == '기타'].shape

(3830, 27)

In [147]:
dd[(dd['상품분류그룹명'] == '기타') & (dd['상품분류명'] == '기타')].shape

(1018, 27)

In [145]:
dd['상품분류그룹명'].isna().sum()

np.int64(2588)

In [165]:
dd['수입자'] = df['수입자'].fillna('해당 없음')
dd['수입국'] = df['수입국'].fillna('해당 없음')

In [166]:
dd.isna().mean()

보상파일번호      0.000000
판정회차        0.000000
사고번호        0.000000
판정일         0.000000
판정결재일       0.000340
판정진행상태      0.000000
사고진행상태      0.000000
수출자         0.000000
보험종목        0.000000
수입국         0.000000
수입자         0.000000
수출보증금액통화    0.000000
수출보증금액      0.000000
사고금액통화      0.000000
외화사고금액      0.000000
사고접수일자      0.000000
보험금청구일      0.000000
판정구분        0.000000
판정금액통화      0.000000
외화판정금액      0.000000
판정사유        0.000000
미화판정금액      0.000000
원화판정금액      0.000000
미화사고금액      0.000000
원화사고금액      0.000000
상품분류명       0.000000
상품분류그룹명     0.000000
사고유형명       0.000000
접수사고유형확정    0.000000
사고설명        0.071830
부보율         0.000000
결제방법        0.042924
결제방법설명      0.047725
결제조건        0.142836
향후결제전망      0.000000
수출자국가       0.000679
dtype: float64

In [ ]:
dd.결제조건.

In [169]:
dd.보험종목.value_counts()

보험종목
단기수출보험(일반수출)          8172
수출신용보증(선적후)           2436
단기수출보험(재판매)           2256
단기수출보험(플러스보험)         1773
단기수출보험(위탁가공무역)        1688
단기수출보험(중계무역)          1310
단기수출보험(글로벌네트워크)       1109
단기수출보험(수출채권유동화)        671
단기수출보험(농수산물패키지)        604
중장기수출보험(구매자신용)         295
농수산물수출보험(패키지형)         150
해외사업금융보험                83
농수산물수출보험(기본형)           23
단기수출보험(단체보험)            18
단기수출보험(포페이팅)            10
단기수출보험(글로벌네트워크탑업)        6
단기수출보험(무역클레임-특약)         5
단기수출보험(구매자신용 일반수출)       4
단기수출보험(다이렉트-선적후)         2
서비스종합보험(기성고 연불방식)        2
수입보험(수입자용)               1
Name: count, dtype: int64

In [171]:
dd[dd.보험종목 == '해외사업금융보험']['수출자국가'].value_counts()

수출자국가
대한민국    43
중국      19
기타       7
Name: count, dtype: int64

In [174]:
dd['수출자국가'] = dd['수출자국가'].fillna('수출자국가 없음')

In [175]:
dd[dd['수출자국가'] == '수출자국가 없음']

,보상파일번호,판정회차,사고번호,판정일,판정결재일,판정진행상태,사고진행상태,수출자,보험종목,수입국,수입자,수출보증금액통화,수출보증금액,사고금액통화,외화사고금액,사고접수일자,보험금청구일,판정구분,판정금액통화,외화판정금액,판정사유,미화판정금액,원화판정금액,미화사고금액,원화사고금액,상품분류명,상품분류그룹명,사고유형명,접수사고유형확정,사고설명,부보율,결제방법,결제방법설명,결제조건,향후결제전망,수출자국가
12806,2013120351810004,1,201000026,2013.08.21,2013.08.21,보상판정 결재완료,사후관리인수인계,신한 외환사업부,해외사업금융보험,중국,CHINA NATIONAL BUILDING MATERIALS AND EQUIPMEN...,USD,572289.37,USD,363212.70,2013-08-20,2013.08.20,지급,USD,345052.06,지급 판정,345052.06,358060522,363212.70,376905818,수출 품목 정보 미제공,수출 품목 그룹 정보 미제공,신용위험-지급불능,신용위험-지급불능,"'10.8.24자 보상심사에 따른 '13.8.29자 제14차 분할원리금(U$363,...",95.0,NaN,NaN,NaN,판단불가,수출자국가 없음
12807,2013120351810003,1,201000025,2013.05.21,2013.05.22,보상판정 결재완료,사후관리인수인계,신한 외환사업부,해외사업금융보험,중국,CHINA NATIONAL BUILDING MATERIALS AND EQUIPMEN...,USD,581653.12,USD,366984.59,2013-05-16,2013.05.16,지급,USD,348635.36,지급 판정,348635.36,361778913,366984.59,380819909,수출 품목 정보 미제공,수출 품목 그룹 정보 미제공,신용위험-지급불능,신용위험-지급불능,'10. 8.24자 보상심사에 따른 '13.5.29자 제 13차 분할원리금(U$36...,95.0,NaN,NaN,NaN,판단불가,수출자국가 없음
12808,2013120351810002,1,201000024,2013.02.20,2013.02.25,보상판정 결재완료,사후관리인수인계,신한 외환사업부,해외사업금융보험,중국,CHINA NORTH WANXING INTERNATIONAL CORP LTD,USD,591548.90,USD,371031.04,2013-02-19,2013.02.20,지급,USD,352479.48,지급 판정,352479.48,365767956,371031.04,385018910,수출 품목 정보 미제공,수출 품목 그룹 정보 미제공,신용위험-지급지체,신용위험-지급지체,'10. 8.24자 보상심사에 따른 '13.2.28자 제 12차 분할원리금(U$37...,95.0,NaN,NaN,NaN,결제불능,수출자국가 없음
13984,2012120351810006,1,201000022,2012.11.28,2012.11.29,보상판정 결재완료,사후관리인수인계,신한 외환사업부,해외사업금융보험,미국,S N S STYLE FABRICS,USD,601657.50,USD,375535.12,2012-11-26,2012.11.27,지급,USD,356758.36,지급 판정,356758.36,370208150,375535.12,389692794,수출 품목 정보 미제공,수출 품목 그룹 정보 미제공,신용위험-지급지체,신용위험-지급지체,'10. 8.24자 보상심사에 따른 '12.11.29자 제 11차 분할원리금(U$3...,95.0,NaN,NaN,NaN,결제불능,수출자국가 없음
13985,2012120351810005,1,201000015,2012.08.29,2012.08.29,보상판정 결재완료,사후관리인수인계,신한 외환사업부,해외사업금융보험,미국,S N S STYLE FABRICS,USD,611446.87,USD,379790.70,2012-08-28,2012.08.28,지급,USD,360801.16,지급 판정,360801.16,374403363,379790.70,394108809,수출 품목 정보 미제공,수출 품목 그룹 정보 미제공,신용위험-지급지체,신용위험-지급지체,'10. 8.24자 보상심사에 따른 '12. 8.29자 제 10차 분할원리금(U$3...,95.0,NaN,NaN,NaN,결제불능,수출자국가 없음
13987,2012120351810003,1,201000014,2012.05.25,2012.05.29,보상판정 결재완료,사후관리인수인계,신한 외환사업부,해외사업금융보험,미국,S N S STYLE FABRICS,USD,14524978.64,USD,383476.98,2012-05-25,2012.05.25,지급,USD,364303.13,지급 판정,364303.13,378037358,383476.98,397934062,수출 품목 정보 미제공,수출 품목 그룹 정보 미제공,신용위험-지급지체,신용위험-지급지체,'12.05.25자 분할원리금 U$383.476.98 미상환 확실,95.0,NaN,NaN,NaN,결제불능,수출자국가 없음
13988,2012120351810001,1,201000011,2012.02.27,2012.02.28,보상판정 결재완료,사후관리인수인계,신한 외환사업부,해외사업금융보험,미국,S N S STYLE FABRICS,USD,14524978.64,USD,388322.50,2012-02-26,2012.02.27,지급,USD,368906.37,지급 판정,368906.37,414908999,388322.50,402962258,수출 품목 정보 미제공,수출 품목 그룹 정보 미제공,신용위험-지급지체,신용위험-지급지체,"'12.02.29자 분할원리금 U$388,322.50 미상환 확실",95.0,NaN,NaN,NaN,결제불능,수출자국가 없음
14974,2011120351810004,1,201000009,2011.11.28,2011.11.28,보상판정 결재완료,사후관리인수인계,신한 외환사업부,해외사업금융보험,브라질,TOPFILME INDUSTRIA DE MATERIAIS PLASTICOS LTDA...,USD,0.00,USD,390531.22,2011-11-25,2011.11.28,지급,USD,371004.65,지급 판정,371004.65,384991525,390531.22,405254246,수출 품목 정보 미제공,수출 품목 그룹 정보 미제공,신용위험-지급지체,신용위험-지급지체,"'11.11.22자 분할원리금 U$390,531.22 미상환 확실",95.0,NaN,NaN,NaN,결제불능,수출자국가 없음
14975,2011120351810003,1,201000007,2011.08.25,2011.08.26,보상판정 결재완료,사후관리인수인계,신한 외환사업부,해외사업금융보험,브라질,P A TEX COMERCIO DE TECIDOS LTDA,USD,0.00,USD,394055.02,2011-08-22,2011.08.22,지급,USD,374352.26,지급 판정,374352.26,388465340,394055.02,408910894,수출 품목 정보 미제공,수출 품목 그룹 정보 미제공,신용위험-지급지체,신용위험-지급지체,"'11.8.29자 분할원리금 U$394,055.02 미상환 확실",95.0,NaN,NaN,NaN,결제불능,수출자국가 없음
14976,2011120351810002,1,201000006,2011.05.27,2011.05.27,보상판정 결재완료,사후관리인수인계,신한 외환사업부,해외사업금융보험,브라질,P A TEX COMERCIO DE TECIDOS LTDA,USD,14009695.60,USD,398431.59,2011-05-20,2011.05.20,지급,USD,378510.01,지급 판정,378510.01,392779837,398431.59,413452460,수출 품목 정보 미제공,수출 품목 그룹 정보 미제공,신용위험-지급지체,신용위험-지급지체,"'11.5.30자 분할원리금 U$398,431.59 미상환 확실",95.0,NaN,NaN,NaN,결제불능,수출자국가 없음


In [180]:
dd.isna().mean()

보상파일번호      0.000000
판정회차        0.000000
사고번호        0.000000
판정일         0.000000
판정결재일       0.000340
판정진행상태      0.000000
사고진행상태      0.000000
수출자         0.000000
보험종목        0.000000
수입국         0.000000
수입자         0.000000
수출보증금액통화    0.000000
수출보증금액      0.000000
사고금액통화      0.000000
외화사고금액      0.000000
사고접수일자      0.000000
보험금청구일      0.000000
판정구분        0.000000
판정금액통화      0.000000
외화판정금액      0.000000
판정사유        0.000000
미화판정금액      0.000000
원화판정금액      0.000000
미화사고금액      0.000000
원화사고금액      0.000000
상품분류명       0.000000
상품분류그룹명     0.000000
사고유형명       0.000000
접수사고유형확정    0.000000
사고설명        0.071830
부보율         0.000000
결제방법        0.000000
결제방법설명      0.004802
결제조건        0.099913
향후결제전망      0.000000
수출자국가       0.000000
dtype: float64

In [179]:
# 세 개 다 결측 → 결제정보 없음
mask_all = (
    dd['결제방법'].isna() & 
    dd['결제방법설명'].isna() & 
    dd['결제조건'].isna()
)
dd.loc[mask_all, ['결제방법', '결제방법설명', '결제조건']] = '결제정보 없음'

In [183]:
dd['결제방법설명'] = dd['결제방법설명'].fillna('결제방법설명 없음')
dd['결제조건'] = dd['결제조건'].fillna('결제조건 없음')

In [178]:
dd[dd['결제방법'].isna()][['결제방법설명', '결제조건']].isna().mean()

결제방법설명    1.0
결제조건      1.0
dtype: float64

In [184]:
# 결제정보가 없는 사고 중 향후결제전망 분포
dd[dd['결제방법'] == '결제정보 없음']['향후결제전망'].value_counts()

향후결제전망
판단불가      636
결제불능      215
기타         31
전매가능        1
일부결제가능      1
결제진행중       1
Name: count, dtype: int64

In [186]:
# 결제조건 없음인 경우 향후전망 분포
dd[dd['결제조건'] == '결제조건 없음']['향후결제전망'].value_counts()


향후결제전망
판단불가      1403
기타         642
결제불능         9
일부결제가능       4
결제가능         2
Name: count, dtype: int64

In [187]:
dd['사고설명'] = dd['사고설명'].fillna('설명 없음')

In [203]:
dd['사고설명'].value_counts()

사고설명
설명 없음                                                                                                                                              1481
수입자 지급지체                                                                                                                                           1112
첨부파일 참조                                                                                                                                             392
- 동일 대표 수입자 : 해당사항 없음.\n- 경합 수출자 : 해당사항 없음.                                                                                                         316
수입자의 지급지체                                                                                                                                           304
                                                                                                                                                   ... 
(주)LG디스플레이 대 독일 LOEWE TECHNOLOGIES GMBH 간 단기수출보험(해외지사별-재판매) 사고건\n- 경합수출자 : 없음      

In [188]:
dd.isna().mean()

보상파일번호      0.00000
판정회차        0.00000
사고번호        0.00000
판정일         0.00000
판정결재일       0.00034
판정진행상태      0.00000
사고진행상태      0.00000
수출자         0.00000
보험종목        0.00000
수입국         0.00000
수입자         0.00000
수출보증금액통화    0.00000
수출보증금액      0.00000
사고금액통화      0.00000
외화사고금액      0.00000
사고접수일자      0.00000
보험금청구일      0.00000
판정구분        0.00000
판정금액통화      0.00000
외화판정금액      0.00000
판정사유        0.00000
미화판정금액      0.00000
원화판정금액      0.00000
미화사고금액      0.00000
원화사고금액      0.00000
상품분류명       0.00000
상품분류그룹명     0.00000
사고유형명       0.00000
접수사고유형확정    0.00000
사고설명        0.00000
부보율         0.00000
결제방법        0.00000
결제방법설명      0.00000
결제조건        0.00000
향후결제전망      0.00000
수출자국가       0.00000
dtype: float64

In [190]:
dd.columns

Index(['보상파일번호', '판정회차', '사고번호', '판정일', '판정결재일', '판정진행상태', '사고진행상태', '수출자',
       '보험종목', '수입국', '수입자', '수출보증금액통화', '수출보증금액', '사고금액통화', '외화사고금액',
       '사고접수일자', '보험금청구일', '판정구분', '판정금액통화', '외화판정금액', '판정사유', '미화판정금액',
       '원화판정금액', '미화사고금액', '원화사고금액', '상품분류명', '상품분류그룹명', '사고유형명', '접수사고유형확정',
       '사고설명', '부보율', '결제방법', '결제방법설명', '결제조건', '향후결제전망', '수출자국가'],
      dtype='object')

In [192]:
dd.판정회차.value_counts()

판정회차
1    18825
2     1410
3      208
7       96
8       48
4       19
5        8
6        4
Name: count, dtype: int64

In [200]:
y = dd.groupby('사고번호')['보상파일번호'].count().reset_index()
y['보상파일번호'].value_counts()

보상파일번호
1    15900
2     1270
3      575
4       65
5       35
6        3
Name: count, dtype: int64

In [196]:
dd.groupby('보상파일번호')['사고번호'].count().value_counts()

사고번호
1      3787
2       950
3       401
4       242
5       151
       ... 
65        1
79        1
63        1
42        1
134       1
Name: count, Length: 68, dtype: int64

In [201]:
dd['판정구분'].value_counts()

판정구분
지급         15769
면책          4498
지급유예         176
가지급          162
기타지급거절        12
보험관계불성립        1
Name: count, dtype: int64

In [202]:
dd['사고유형명'].value_counts()

사고유형명
신용위험-지급지체                         15941
신용위험-파산                            1417
신용위험-지급불능                          1049
수입국검역위험                             572
기타(워크아웃 등 기업구조조정 개시)                487
신용위험-지급거절                           456
신용위험-인수거절                           186
비상위험-지급국에 원인이 있는 송금지연               175
비상위험-외국에서 실시되는 환거래의 제한 또는 금지        151
가격상승위험                               57
비상위험-전쟁 등으로 인한 환거래 불능                36
신용위험-인수불능                            34
비상위험-송금을 지연시키는 정치적 사건, 경제적 어려움       27
클레임비용위험                              10
비상위험-전쟁등으로 인한 수입국의 수입불능              10
비상위험-외국에서 발생한 사유로 인한 수송불능             6
물품의 품질 및 수량문제(무신용장거래)                 3
신용위험-선급금 환급지체                         1
Name: count, dtype: int64

In [207]:
dd['사고진행상태'].value_counts()

사고진행상태
사후관리인수인계     14609
면책종결          2062
기타종결           918
보험금지급          895
지급심사완료         853
전액회수종결         687
이의신청 심의완료      350
사후관리 반송         91
사후관리인수인계중       90
이의신청중           50
지급심사중           13
Name: count, dtype: int64

In [ ]:
# 의미 없는 텍스트 정의 (확장 가능)
meaningless_texts = ['설명 없음', '첨부파일 참고', '첨부파일 참조', '별첨 참고', '없음', '첨부파일', '참조']

# 모두 소문자 처리 후 공백 제거
meaningless_texts = [t.strip().lower() for t in meaningless_texts]

# 사고설명 전처리 (소문자 + 공백제거)
desc_clean = dd['사고설명'].fillna('').str.strip().str.lower()

# 의미 있는 설명만 추출
mask_meaningful = ~desc_clean.isin(meaningless_texts)
mask_meaningful


0        True
1        True
2        True
3        True
4        True
         ... 
21766    True
21767    True
21768    True
21770    True
21772    True
Name: 사고설명, Length: 20618, dtype: bool

In [213]:
# 의미 없는 텍스트 정의 (전처리 대비용)
meaningless_texts = ['설명 없음', '첨부파일 참고', '첨부파일 참조', '별첨 참고', '없음', '첨부파일', '참조']
meaningless_texts = [t.strip().lower() for t in meaningless_texts]

# 사고설명 전처리
desc_clean = dd['사고설명'].fillna('').str.strip().str.lower()

# 조건 1: 의미 없는 텍스트 제외
cond1 = ~desc_clean.isin(meaningless_texts)

# 조건 2: '첨부' 또는 '참고' 포함 안 된 경우
cond2 = ~desc_clean.str.contains('첨부|참고', na=False)

# 조건 3: 글자 수가 15자 이상
cond3 = desc_clean.str.len() >= 15

# 최종 유의미한 설명 조건
mask_meaningful = cond1 & cond2 & cond3
df_meaningful = dd[mask_meaningful]

# 통계
total = len(df)
meaningful = len(df_meaningful)
unique_meaningful = df_meaningful['사고설명'].nunique()

print(f"✅ 유의미한 사고설명 비율: {meaningful / total:.2%} ({meaningful} / {total})")
print(f"✅ 유의미한 설명 중 고유 설명 수: {unique_meaningful}")


✅ 유의미한 사고설명 비율: 59.15% (13780 / 23296)
✅ 유의미한 설명 중 고유 설명 수: 4452


### design read & 데이터 속성 파악

In [2]:
df = pd.read_csv('data/design.csv', encoding='cp949')

In [3]:
pd.set_option('display.max_columns', None)
df.sample(1)

,보상파일번호,판정회차,사고번호,판정일,판정결재일,판정진행상태,사고진행상태,수출자,보험종목,수입국,수입자,수출보증금액통화,수출보증금액,사고금액통화,외화사고금액,사고접수일자,보험금청구일,판정구분,판정금액통화,외화판정금액,판정사유,미화판정금액,원화판정금액,미화사고금액,원화사고금액,상품분류명,상품분류그룹명,사고유형명,접수사고유형확정,사고설명,부보율,결제방법,결제방법설명,결제조건,향후결제전망,수출자국가
16593,2020120351210001,1,151020870,2021.07.23,2021.07.23,보상판정 결재완료,면책종결,(주)뉴티스,수출신용보증(선적후),미국,SH USA INC,USD,3213.87,USD,3213.87,2020-01-14,2020.03.16,면책,USD,3181.73,주의의무 해태로 인한 손실가중,3181.73,3405405,3213.87,3439805,가. 리드프레임,전기 및 전자제품,신용위험-지급지체,신용위험-지급지체,(주)뉴티스 대 중국 KWANG MYUNG SEMICONDUCTOR TIANJIN ...,99.0,O/A(T/T 포함),결제방법(인수),월말 마감후,판단불가,대한민국


In [5]:
df['판정구분'].value_counts()

판정구분
지급         15769
면책          4498
지급유예         176
가지급          162
기타지급거절        12
보험관계불성립        1
Name: count, dtype: int64

In [7]:
df[df['판정구분'] == '보험관계불성립']

,보상파일번호,판정회차,사고번호,판정일,판정결재일,판정진행상태,사고진행상태,수출자,보험종목,수입국,수입자,수출보증금액통화,수출보증금액,사고금액통화,외화사고금액,사고접수일자,보험금청구일,판정구분,판정금액통화,외화판정금액,판정사유,미화판정금액,원화판정금액,미화사고금액,원화사고금액,상품분류명,상품분류그룹명,사고유형명,접수사고유형확정,사고설명,부보율,결제방법,결제방법설명,결제조건,향후결제전망,수출자국가
20267,2013120351400032,1,111025335,2014.02.10,2014.02.11,보상판정 결재완료,사후관리인수인계,농업회사법인 (주) 강원무역,단기수출보험(농수산물패키지),영국,BLACKSTAR AMPLIFICATION LTD,KRW,300000000.0,KRW,142473080.0,2013-10-11,2013.11.14,보험관계불성립,KRW,3276090.0,"적용대상거래 및 보험계약의 성립(부보대상거래, 조건부 신용장, 대금지급책임 면제 등)",2903.82,3276090,126283.53,142473080,수출 품목 정보 미제공,수출 품목 그룹 정보 미제공,신용위험-지급지체,신용위험-지급지체,수입자 지급지체,100.0,D/A,결제방법(인수),결제조건 없음,판단불가,대한민국


In [6]:
df['판정사유'].value_counts()

판정사유
지급 판정                                              15764
보험계약자의 고의 또는 과실로 인하여 발생한 손실                          979
연속수출                                                 734
보상한도를 초과하는 손실                                        426
보험관계의 성립                                             397
주의의무 해태로 인한 손실가중                                     385
신용보증조건 위반                                            286
신용보증관계의 성립                                           236
권리보전의무 해태로 인한 손실가중                                   231
적용대상 수출거래                                            215
지급유예 판정                                              174
변제충당                                                 141
보험계약의 해지(고지의무, 내용변경, 보험료 미납)                         120
보험계약자와 수출계약상대방간에 분쟁발생                                 84
지급할 보험금을 산정하기 위하여 장기간이 소요되는 경우                        70
보증채무의 범위                                              44
사고발생통지 의무                                             42
신용보증대상 수출거래               

In [8]:
df[df['판정사유'] == '보험계약자와 재판매계약상대방간에 분쟁발생']

,보상파일번호,판정회차,사고번호,판정일,판정결재일,판정진행상태,사고진행상태,수출자,보험종목,수입국,수입자,수출보증금액통화,수출보증금액,사고금액통화,외화사고금액,사고접수일자,보험금청구일,판정구분,판정금액통화,외화판정금액,판정사유,미화판정금액,원화판정금액,미화사고금액,원화사고금액,상품분류명,상품분류그룹명,사고유형명,접수사고유형확정,사고설명,부보율,결제방법,결제방법설명,결제조건,향후결제전망,수출자국가
20439,2010120351360005,1,111019281,2010.09.13,2010.09.14,보상판정 결재완료,사후관리인수인계,삼성전자(주),단기수출보험(글로벌네트워크),영국,MARSHALL AMPLIFICATION P L C (MARSHALL / MARSH...,USD,165517.12,USD,513885.45,2010-04-01,2010.08.02,가지급,USD,292914.7,보험계약자와 재판매계약상대방간에 분쟁발생,292914.7,346049426,513885.45,607104270,전기식의 것,운송장비 및 부품,신용위험-지급지체,신용위험-지급지체,설명 없음,95.0,NET,결제방법(인수),Days From B/L Date,판단불가,대한민국


In [8]:
df.상품분류명.value_counts().head(30)

상품분류명
기타                                              3830
수출 품목 정보 미제공                                    2588
강력사(폴리에스테르의 것)                                   926
전기식의 것                                           745
래디알구조의 것                                         690
앰프리시버                                            548
고무제의 공기타이어                                       420
탄성사의 함유중량이 전 중량의 100분의 5이상이며, 고무사는 함유하지 않은 것     394
의류                                               374
염색한 것                                            262
인쇄용 또는 필기용 지와 판지                                 218
합성섬유제의 것                                         217
나. 기타                                            186
컴퓨터 케이스                                          182
드레스                                              180
날염한 것                                            172
김                                                171
폴리프로필렌                                           165
기타                                      

In [7]:
df.결제조건.value_counts()

결제조건
Days From B/L Date                            9207
Days After B/L Date                           3610
결제조건 없음                                       2060
Days After Invoice Date                       1666
결제정보 없음                                        885
Days From Invoice Date                         658
Days After Sight                               612
월말 마감후                                         487
Days After Arrival                             343
After Delivery Date(Net거래로 수입지인도일 기준 보험기산)     342
Days From Nego Date                            253
After Finding Docs                             129
At Sight                                       110
Days After Nego Date                            92
Days From Arrival                               81
After Delivery Date(국내수출일 기준 보험기산)              25
At                                              25
On Arrival Of Goods                             15
매월 15일자 마감후                                     12
기타                        

In [215]:
df.isna().mean()

보상파일번호      0.00000
판정회차        0.00000
사고번호        0.00000
판정일         0.00000
판정결재일       0.00034
판정진행상태      0.00000
사고진행상태      0.00000
수출자         0.00000
보험종목        0.00000
수입국         0.00000
수입자         0.00000
수출보증금액통화    0.00000
수출보증금액      0.00000
사고금액통화      0.00000
외화사고금액      0.00000
사고접수일자      0.00000
보험금청구일      0.00000
판정구분        0.00000
판정금액통화      0.00000
외화판정금액      0.00000
판정사유        0.00000
미화판정금액      0.00000
원화판정금액      0.00000
미화사고금액      0.00000
원화사고금액      0.00000
상품분류명       0.00000
상품분류그룹명     0.00000
사고유형명       0.00000
접수사고유형확정    0.00000
사고설명        0.00000
부보율         0.00000
결제방법        0.00000
결제방법설명      0.00000
결제조건        0.00000
향후결제전망      0.00000
수출자국가       0.00000
dtype: float64

In [216]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20618 entries, 0 to 20617
Data columns (total 36 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   보상파일번호    20618 non-null  object 
 1   판정회차      20618 non-null  int64  
 2   사고번호      20618 non-null  int64  
 3   판정일       20618 non-null  object 
 4   판정결재일     20611 non-null  object 
 5   판정진행상태    20618 non-null  object 
 6   사고진행상태    20618 non-null  object 
 7   수출자       20618 non-null  object 
 8   보험종목      20618 non-null  object 
 9   수입국       20618 non-null  object 
 10  수입자       20618 non-null  object 
 11  수출보증금액통화  20618 non-null  object 
 12  수출보증금액    20618 non-null  float64
 13  사고금액통화    20618 non-null  object 
 14  외화사고금액    20618 non-null  float64
 15  사고접수일자    20618 non-null  object 
 16  보험금청구일    20618 non-null  object 
 17  판정구분      20618 non-null  object 
 18  판정금액통화    20618 non-null  object 
 19  외화판정금액    20618 non-null  float64
 20  판정사유      20618 non-null  ob

In [217]:

print('전체 국가 수:', df['수입국'].nunique())

# 상위 20개국 출력
print('\n상위 20개국:')
print(df['수입국'].value_counts().head(20))

# 하위 국가들 (5건 이하) 분석
bottom = df['수입국'].value_counts()
low_count = (bottom <= 5).sum()
print(f'\n하위 국가들 (5건 이하):')
print(f'5건 이하 국가 수: {low_count}개')

전체 국가 수: 153

상위 20개국:
수입국
미국            3789
중국            1730
브라질           1639
일본            1129
독일             749
영국             710
러시아            596
인도             577
베트남            515
이탈리아           488
홍콩             462
아랍에미리트 연합      421
튀르키예(구 터키)     409
인도네시아          366
스페인            328
대만             320
멕시코            299
프랑스            280
태국             278
콜롬비아           242
Name: count, dtype: int64

하위 국가들 (5건 이하):
5건 이하 국가 수: 39개
